In [1]:
import findspark
import re
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
import pandas as pd
from nltk.corpus import stopwords
import nltk 
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder,StringIndexer, VectorAssembler
import numpy as np
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.linalg import Vector as MllibVector, Vectors as MLLibVectors

In [2]:
sc = pyspark.SparkContext()

In [3]:
spark = SparkSession \
    .builder \
    .appName("PA3") \
    .config("spark.some.config.option","some-value") \
    .getOrCreate()

In [4]:
#read in data
df = pd.read_csv("train.csv")
df = spark.createDataFrame(df)

In [5]:
df =df.withColumn('plot',lower(regexp_replace(df['plot'],"[^a-zA-Z ]","")))

In [6]:
regexToken = RegexTokenizer(inputCol='plot',outputCol='tokens', pattern=" ")

In [7]:
df = regexToken.transform(df)

In [8]:
stop = stopwords.words('english')

In [9]:
stopWordsRemover = StopWordsRemover(inputCol = 'tokens',outputCol = 'cleaned').setStopWords(stop)

In [10]:
df = stopWordsRemover.transform(df)

In [11]:
counts = CountVectorizer(inputCol= 'cleaned',outputCol= 'features',minDF=10)

In [12]:
model = counts.fit(df)

In [13]:
df = model.transform(df)

In [14]:
 df =df.drop('plot','tokens')

In [15]:
labels = df.select('genre')

In [16]:
labels= labels.rdd.flatMap(lambda x:x)

In [17]:
feat = df.select('features')
feat = feat.rdd.flatMap(lambda x:x)

In [18]:
feats = feat.collect()

In [19]:
movie_ids = df.select('movie_id')
movie_ids = movie_ids.rdd
movie_ids = movie_ids.flatMap(lambda x:x)
movie_ids = movie_ids.collect()


In [20]:
keylist = pd.read_csv('mapping.csv')
keylist.rename(columns={'Unnamed: 0': 'i'}, inplace=True)
keylist = spark.createDataFrame(keylist)
k = keylist.select('0')
k1 = keylist.select('i')
k = k.rdd.flatMap(lambda x:x)
k1 = k1.rdd.flatMap(lambda x:x)
genre_list = k1.zip(k)

In [21]:
def test(data):
    ans = []
    for i in range(len(data)):
        val = 0.0
        labels = data[i][0].replace('[','').replace(']','').replace('\'','').split(',')
        for lab in labels:
            if lab == label:
                val = 1.0
                break
        pt = LabeledPoint(val, MLLibVectors.sparse(data[i][1].size,data[i][1].indices,data[i][1].values))
        ans.append(pt)
    return ans

In [22]:
def makeClassifier(data):
    lst = test(data)
    return SVMWithSGD.train(sc.parallelize(lst), iterations=100)

In [23]:
lab2sparse = labels.zip(feat)

In [24]:
lab2sparse.take(1)

[("['World cinema', 'Drama']",
  SparseVector(24527, {126: 1.0, 180: 1.0, 499: 1.0, 536: 1.0, 576: 1.0, 1867: 1.0, 2074: 1.0, 3127: 1.0, 5254: 1.0, 8209: 1.0, 13154: 1.0, 21390: 1.0, 21912: 1.0}))]

In [25]:
l2sm = lab2sparse.map(lambda x: (x[0],x[1]))
data = l2sm.collect()

In [26]:
data[0]

("['World cinema', 'Drama']",
 SparseVector(24527, {126: 1.0, 180: 1.0, 499: 1.0, 536: 1.0, 576: 1.0, 1867: 1.0, 2074: 1.0, 3127: 1.0, 5254: 1.0, 8209: 1.0, 13154: 1.0, 21390: 1.0, 21912: 1.0}))

In [27]:
classifiers = []

for label in k.collect():
    print('Trained', label, 'Classifier')
    classifiers.append(makeClassifier(data))

Trained Drama Classifier


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 34468)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
Traceback (most recent call last):
 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:41677)
Traceback (most recent call last):
  File "/home/cse587/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-4da37e29aa38>", line 5, in <module>
    classifiers.append(makeClassifier(data))
  File "<ipython-input-22-66aee14af7df>", line 3, in makeClassifier
    return SVMWithSGD.train(sc.parallelize(lst), iterations=100)
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/classification.py", line 557, in train
    return _regression_train_wrapper(train, SVMModel, data, initialWeights)
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/regression.py", line 220, in _regression_train_wrapper
    weights, intercept = train_func(data, _convert_to_vector(initial_weights))
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/c

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:41677)
Traceback (most recent call last):
  File "/home/cse587/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-4da37e29aa38>", line 5, in <module>
    classifiers.append(makeClassifier(data))
  File "<ipython-input-22-66aee14af7df>", line 3, in makeClassifier
    return SVMWithSGD.train(sc.parallelize(lst), iterations=100)
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/classification.py", line 557, in train
    return _regression_train_wrapper(train, SVMModel, data, initialWeights)
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/regression.py", line 220, in _regression_train_wrapper
    weights, intercept = train_func(data, _convert_to_vector(initial_weights))
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/c

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:41677)
Traceback (most recent call last):
  File "/home/cse587/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-4da37e29aa38>", line 5, in <module>
    classifiers.append(makeClassifier(data))
  File "<ipython-input-22-66aee14af7df>", line 3, in makeClassifier
    return SVMWithSGD.train(sc.parallelize(lst), iterations=100)
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/classification.py", line 557, in train
    return _regression_train_wrapper(train, SVMModel, data, initialWeights)
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/regression.py", line 220, in _regression_train_wrapper
    weights, intercept = train_func(data, _convert_to_vector(initial_weights))
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/c

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:41677)
Traceback (most recent call last):
  File "/home/cse587/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-4da37e29aa38>", line 5, in <module>
    classifiers.append(makeClassifier(data))
  File "<ipython-input-22-66aee14af7df>", line 3, in makeClassifier
    return SVMWithSGD.train(sc.parallelize(lst), iterations=100)
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/classification.py", line 557, in train
    return _regression_train_wrapper(train, SVMModel, data, initialWeights)
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/regression.py", line 220, in _regression_train_wrapper
    weights, intercept = train_func(data, _convert_to_vector(initial_weights))
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/c

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:41677)
Traceback (most recent call last):
  File "/home/cse587/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-4da37e29aa38>", line 5, in <module>
    classifiers.append(makeClassifier(data))
  File "<ipython-input-22-66aee14af7df>", line 3, in makeClassifier
    return SVMWithSGD.train(sc.parallelize(lst), iterations=100)
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/classification.py", line 557, in train
    return _regression_train_wrapper(train, SVMModel, data, initialWeights)
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/regression.py", line 220, in _regression_train_wrapper
    weights, intercept = train_func(data, _convert_to_vector(initial_weights))
  File "/home/cse587/spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/c

Py4JError: An error occurred while calling o327.trainSVMModelWithSGD

In [ ]:
ans = {}
for i in range(len(sparseVectorsList_Test)):
    res = []
    movie_id = movie_ids[i]
    sv = sparseVectorsList_Test[i]
    for classifier in classifiers:
        res.append(classifier.predict(sv))
    ans[movie_id] = res